In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.applications import MobileNet
from keras.layers import Dense, Flatten
from keras.models import Sequential, load_model
from sklearn.preprocessing import LabelBinarizer
from PIL import Image
import numpy as np

Using TensorFlow backend.


In [2]:
channels = 3
img_size = 32

data = pd.read_csv("../input/digit-recognizer/train.csv")
data.label = data.label.astype('category')

mobile = MobileNet(include_top=False, weights='imagenet', input_shape=(img_size,img_size,3))

/opt/conda/lib/python3.6/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


17227776/17225924 [==============================] - 0s 0us/step


In [3]:
valset = data[data.label==0][:32]
for i in range(1,10):
    valset = pd.concat([valset, data[data.label==i][:32]], axis=0)
data = data.drop(valset.index, axis=0)

subset = data[data.label==0][:32]
for i in range(1,10):
    subset = pd.concat([subset, data[data.label==i][:32]], axis=0)

In [4]:
y_train = subset.label
y_val = valset.label
subset = subset.drop(columns=['label'])
valset = valset.drop(columns=['label'])

encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)

subset = np.reshape(np.array(subset), newshape=(32*10,28,28))
valset = np.reshape(np.array(valset), newshape=(32*10,28,28))

In [5]:
def convert_resize(data):
    new_data = []
    for i in range(len(data)):
        img = Image.fromarray(data[i].astype('uint8'), 'L')
        img = img.convert('RGB').resize((32,32), Image.ANTIALIAS)
        new_data.append(np.array(img))
    return np.array(new_data)

In [6]:
# Scale to 0-1
new_subset = convert_resize(subset)/255
new_valset = convert_resize(valset)/255

In [7]:
def trfModel():
    model = Sequential()
    model.add(mobile)
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
model = trfModel()
model.fit(x=new_subset, y=y_train, batch_size=32, epochs=10)

Epoch 1/10
320/320 [==============================] - 9s 27ms/step - loss: 2.4462 - acc: 0.3688
Epoch 2/10
320/320 [==============================] - 0s 1ms/step - loss: 1.0270 - acc: 0.7250
Epoch 3/10
320/320 [==============================] - 0s 1ms/step - loss: 0.6501 - acc: 0.8000
Epoch 4/10
320/320 [==============================] - 0s 1ms/step - loss: 0.4273 - acc: 0.8594
Epoch 5/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2774 - acc: 0.9344
Epoch 6/10
320/320 [==============================] - 0s 1ms/step - loss: 0.3035 - acc: 0.9125
Epoch 7/10
320/320 [==============================] - 0s 1ms/step - loss: 0.3023 - acc: 0.9094
Epoch 8/10
320/320 [==============================] - 0s 1ms/step - loss: 0.2681 - acc: 0.9250
Epoch 9/10
320/320 [==============================] - 0s 1ms/step - loss: 0.1969 - acc: 0.9281
Epoch 10/10
320/320 [==============================] - 0s 1ms/step - loss: 0.1854 - acc: 0.9437


In [9]:
model.evaluate(x=new_valset, y=encoder.transform(y_val))

320/320 [==============================] - 1s 2ms/step


[0.9111291021108627, 0.809375]

In [10]:
full_y_train = encoder.transform(data.label)
data = data.drop(columns=['label'])
data = np.reshape(np.array(data), newshape=(len(data), 28, 28))

full_x_train = convert_resize(data)/255

In [11]:
full_model = trfModel()
full_model.fit(x=full_x_train, y=full_y_train, batch_size=32, epochs=10)

Epoch 1/10
41680/41680 [==============================] - 53s 1ms/step - loss: 0.2995 - acc: 0.9233
Epoch 2/10
41680/41680 [==============================] - 48s 1ms/step - loss: 0.1233 - acc: 0.9685
Epoch 3/10
41680/41680 [==============================] - 47s 1ms/step - loss: 0.1192 - acc: 0.9716
Epoch 4/10
41680/41680 [==============================] - 47s 1ms/step - loss: 0.1673 - acc: 0.9614
Epoch 5/10
41680/41680 [==============================] - 47s 1ms/step - loss: 0.2535 - acc: 0.9478
Epoch 6/10
41680/41680 [==============================] - 46s 1ms/step - loss: 0.2667 - acc: 0.9532
Epoch 7/10
41680/41680 [==============================] - 46s 1ms/step - loss: 0.0985 - acc: 0.9763
Epoch 8/10
41680/41680 [==============================] - 47s 1ms/step - loss: 0.0644 - acc: 0.9849
Epoch 9/10
41632/41680 [============================>.] - ETA: 0s - loss: 0.0769 - acc: 0.9819

In [12]:
full_model.evaluate(x=new_valset, y=encoder.transform(y_val))

320/320 [==============================] - 1s 4ms/step


[0.18138439753092825, 0.95625]

## GAN + Transfer Learning

In [13]:
generator = load_model("../input/conditional-dcgan-cnn-using-32-images/generator.h5")

In [14]:
conditions = []
for i in range(10):
    condition = np.zeros(10, dtype='int')
    condition[i] = 1
    conditions.append([condition for z in range(3200)])

conditions = np.reshape(np.array(conditions), newshape=(3200*10,10))
noises = np.random.normal(0,1, size=(3200*10,100))
synthetic_images = generator.predict([noises, conditions])

In [15]:
synthetic_images = synthetic_images*255
synthetic_images = np.reshape(synthetic_images, newshape=(3200*10, 28, 28))
reshaped_synthetic_images = convert_resize(synthetic_images)/255

In [16]:
synthetic_model = trfModel()
synthetic_model.fit(x=reshaped_synthetic_images, y=conditions, batch_size=32, epochs=10)

Epoch 1/10
32000/32000 [==============================] - 43s 1ms/step - loss: 0.1319 - acc: 0.9807
Epoch 2/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0459 - acc: 0.9936
Epoch 3/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0349 - acc: 0.9937
Epoch 4/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0064 - acc: 0.9981
Epoch 5/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0145 - acc: 0.9966
Epoch 6/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0271 - acc: 0.9944
Epoch 7/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0239 - acc: 0.9951
Epoch 8/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0062 - acc: 0.9981
Epoch 9/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0050 - acc: 0.9983
Epoch 10/10
32000/32000 [==============================] - 36s 1ms/step - loss: 0.0223 - acc: 0.9952

In [17]:
synthetic_model.evaluate(x=new_valset, y=encoder.transform(y_val))

320/320 [==============================] - 2s 6ms/step


[0.2700042124837637, 0.91875]